In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

### CUDA语义的支持

In [2]:
#使用torch.device指定设备,Pytorch默认使用cpu设备
print("Default Device:{}".format(torch.Tensor([4,5,6]).device))
#cpu设备可以使用"cpu:0"来指定
device = torch.Tensor([1,2,3],device="cpu:0").device
print("Device Type:{}".format(device))
#用torch.device指定cpu:0设备
cpu1 =torch.device("cpu:0")
print("CPU Device:【{}:{}】".format(cpu1.type,cpu1.index))
#使用索引的方式，默认设备为CUDA设备
gpu =torch.device(0)
print("GPU Device:【{}:{}】".format(gpu.type,gpu.index))
#也可以通过torch.device("cuda:0")的方式指定使用哪一块GPU
gpu =torch.device("cuda:0")
print("GPU Device:【{}:{}】".format(gpu.type,gpu.index))
#查看所有可用GPU个数
print("Total GPU Count:{}".format(torch.cuda.device_count()))
#获取系统CPU数量
print("Total CPU Count:{}".format(torch.cuda.os.cpu_count()))
#获取GPU设备名称
print(torch.cuda.get_device_name(torch.device("cuda:0")))
#GPU设备是否可用
print("GPU Is Available:{}".format(torch.cuda.is_available()))

Default Device:cpu
Device Type:cpu
CPU Device:【cpu:0】
GPU Device:【cuda:0】
GPU Device:【cuda:0】
Total GPU Count:1
Total CPU Count:12
GeForce GTX 1060
GPU Is Available:True


### 将Tensor从CPU转移到GPU并进行计算

In [3]:
cpu_tensor = torch.Tensor([[1,4,7],[3,6,9],[2,5,8]])
print(cpu_tensor.device)
#使用to()方法将cup_tensor转到GPU上
gpu_tensor1 = cpu_tensor.to(torch.device("cuda:0"))
print(gpu_tensor1.device)
#使用cuda()方法将cpu_tensor转移到GPU上
gpu_tensor2 = cpu_tensor.cuda(torch.device("cuda:0"))
print(gpu_tensor2.device)
#使用copy_()方法将cpu_tensor转移到GPU上
gpu_tensor3 = cpu_tensor.copy_(gpu_tensor2)
print(gpu_tensor2.device)
print(gpu_tensor3**3)

cpu
cuda:0
cuda:0
cuda:0
tensor([[  1.,  64., 343.],
        [ 27., 216., 729.],
        [  8., 125., 512.]])


### 直接在GPU上创建Tensor

In [4]:

#使用torch.tensor方法，注意和torch.Tensor的区别,前者可通过device指定GPU设备，而后者只能在CPU上创建。
gpu_tensor1 = torch.tensor([[2,5,8],[1,4,7],[3,6,9]],device=torch.device("cuda:0"))
print(gpu_tensor1.device)
#Tensor方法device只能是CPU设备，否则报错。在CPU上创建后可使用copy_()、cuda()、to()等方法转移到GPU上
cpu_tensor = torch.Tensor([[2,5,8],[1,4,7],[3,6,9]],device=torch.device("cpu:0"))
print(cpu_tensor.device)
#除了Tensor方法外，torch上的其它例如rand、randn等方法都可以通过device参数指定GPU设备
print(torch.rand((3,4),device=torch.device("cuda:0")))
print(torch.zeros((2,5),device=torch.device("cuda:0")))

cuda:0
cpu
tensor([[0.8562, 0.4634, 0.5491, 0.5583],
        [0.6402, 0.0205, 0.4074, 0.0257],
        [0.8147, 0.7708, 0.5986, 0.5964]], device='cuda:0')
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], device='cuda:0')


### CUDA Stream

In [5]:
cuda = torch.device('cuda')
#创建新的Stream
s = torch.cuda.Stream()
#使用默认Stream
A = torch.randn((1, 10), device=cuda)
print("Current Stream:{}".format(torch.cuda.current_stream()))
for i in range(10):
    #在新的Stream上对默认Stream上创建的Tenssor求和
    with torch.cuda.stream(s):
        print("Current Stream:{}".format(torch.cuda.current_stream()))
        # sum() may start execution before randn() finishes!
        B = torch.sum(A)

Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0daf0>


In [6]:
cuda = torch.device('cuda')
#创建新的Stream
s = torch.cuda.Stream()
#使用默认Stream
A = torch.randn((100000, 7000), device=cuda)
default_stream = torch.cuda.current_stream()
print("Default Stream:{}".format(default_stream))
#等待创建A的stream完成
torch.cuda.Stream.synchronize(default_stream.cuda_stream)
#在新的Stream上对默认Stream上创建的Tenssor求和
with torch.cuda.stream(s):
    for i in range(5):
        print("Current Stream:{}".format(torch.cuda.current_stream()))
        B = torch.sum(A)

Default Stream:<torch.cuda.Stream device=0 cuda_stream=0x0>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0dd30>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0dd30>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0dd30>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0dd30>
Current Stream:<torch.cuda.Stream device=0 cuda_stream=0x1eae6e0dd30>


In [15]:
#释放没有使用的缓存数据
torch.cuda.empty_cache()

In [19]:
#获取缓存数据大小
torch.cuda.memory_cached()

2802188288

In [23]:
#最大缓存大小
torch.cuda.max_memory_cached()

2803236864

In [22]:
#最大分配内存大小
torch.cuda.max_memory_allocated()

2801129984

### 固定内存缓冲区

In [25]:
# pytorch中提出了固定缓冲区的概念，当从CPU设备的固定缓冲区(页面锁定)拷贝数据到GPU时，速度会有很大提升。
# 普通tensor上调用pin_memory方法即将tensor放入固定缓冲区中
x = torch.Tensor([[1,2,4],[5,7,9],[3,7,10]])
y = torch.pin_memory(x)
#或者x直接调用pin_memory方法
z = x.pin_memory()
#需要注意：pin_memory方法对应的拷贝，因此内存地址是不一样的
print("id:{}".format(id(x)))
print("id:{}".format(id(y)))
print("id:{}".format(id(z)))
#一旦将tensor放入固定缓冲区，可以使用asynchrinize方式拷贝数据到GPU，例如使用cuda()方法，将关键字参数non_blocking设置为True
a = z.cuda(non_blocking=True)
print(a)
print("is_pinned:{}/{}".format(x.is_pinned(),z.is_pinned()))

id:2858737070368
id:2858735080432
id:2858737070440
tensor([[ 1.,  2.,  4.],
        [ 5.,  7.,  9.],
        [ 3.,  7., 10.]], device='cuda:0')
is_pinned:False/True


In [17]:
#在数据加载器Dataloader中指定pin_memory关键字参数为true

In [38]:
#从视觉库中导入数据集及图片数据转化方法
from torchvision import datasets, transforms
#穿件Dataset实例
mnist_dataset = datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1,), (0.3,))
                    ]))
#创建DataLoader,加载MNIST数据集
data_loader = torch.utils.data.DataLoader(mnist_dataset,batch_size=32, shuffle=True, num_workers=10,**{"pin_memory":True})

In [64]:
for data,indx in data_loader:
    i = i+1
    print(data[0].is_pinned(),indx.is_pinned())
    #将数据拷贝到GPU
    data.to(torch.device("cuda"),non_blocking=True)
    indx.to(torch.device("cuda"),non_blocking=True)

True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True
True True


### 自动设备感知

In [78]:
#创建默认的CPU设备
device = torch.device("cpu")
#如果GPU设备可用，将默认热备改为GPU
if(torch.cuda.is_available()):
    device = torch.device("cuda")
#使用tensor方法，device关键字参数指定设备
a = torch.tensor([1,2,3],device=device)
print(a)

tensor([1, 2, 3], device='cuda:0')


In [102]:
#将网络迁移到GPU
import torch.nn
class LinearRegression(torch.nn.Module):
    def __init__(self):
        super(LinearRegression,self).__init__()
        #输入输出都是一维
        self.linear = torch.nn.Linear(1,1)
    def forward(self,x):
        return self.linear(x)
regression = LinearRegression().to(device=device)
for para in regression.parameters():
    print(para)

Parameter containing:
tensor([[0.1795]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.8931], device='cuda:0', requires_grad=True)


Parameter containing:
tensor([[0.5079]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.9685], device='cuda:0', requires_grad=True)
